In [45]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    # df = pd.DataFrame(res_rows)
    return res_rows

INFO:root:connection created


In [53]:
get_list_sql = """
    select interact_account_address
        ,sum(IFNULL(transfer_in_dollar_amt,0) + IFNULL(transfer_out_dollar_amt,0)) as total_amt
    from dm.dm_acct_eoa_interaction_aggr_di
    where is_eoa_address = 1
    group by 1
    order by total_amt desc
    limit 1000
"""
res = getSqlResult(get_list_sql)

In [54]:
alameda_address_list = [
"0x073dca8acbc11ffb0b5ae7ef171e4c0b065ffa47",
"0x882a812d75aee53efb8a144f984b258b6c4807f0",
"0xbefe4f86f189c1c817446b71eb6ac90e3cb68e60",
"0xb78e90e2ec737a2c0a24d68a0e54b410fff3bd6b",
"0x964d9d1a532b5a5daeacbac71d46320de313ae9c",
"0xfa453aec042a837e4aebbadab9d4e25b15fad69d",
"0x4deb3edd991cfd2fcdaa6dcfe5f1743f6e7d16a6",
"0x477573f212a7bdd5f7c12889bd1ad0aa44fb82aa",
"0xce31190a03fc3c5f23167e88e75066824823222d",
"0x73c0ae50756c7921d1f32ada71b8e50c5de7ff9c",
"0x712d0f306956a6a4b4f9319ad9b9de48c5345996",
"0x60ae578abdfded1fb0555f54148fdd7b400a34ed",
"0x0c0fe4e0236480e16b679ee1fd0c5247f9cf35f0",
"0x0f4ee9631f4be0a63756515141281a3e2b293bbe",
"0x97137466bc8018531795217f0ecc4ba24dcba5c1",
"0x84d34f4f83a87596cd3fb6887cff8f17bf5a7b83",
"0x78835265ac857bf3420830c71987b1a55f73c2dc",
"0x4c8cfe078a5b989cea4b330197246ced82764c63",
"0x93c08a3168fc469f3fc165cd3a471d19a37ca19e",
"0xca436e14855323927d6e6264470ded36455fc8bd",
"0x83a127952d266a6ea306c40ac62a4a70668fe3bd",
"0xc5ed2333f8a2c351fca35e5ebadb2a82f5d254c3",
"0x89183c0a8965c0299997be9af700a801bdccc2da",
"0xe5d0ef77aed07c302634dc370537126a2cd26590",
"0x5d13f4bf21db713e17e04d711e0bf7eaf18540d6",
"0x3507e4978e0eb83315d20df86ca0b976c0e40ccb",
"0xa726c00cda1f60aaab19bc095d02a46556837f31",
"0x345c9ae61ff6e68b319d61082a017ba854892488",
"0x544c42fbb96b39b21df61cf322b5edc285ee7429",
"0xa6e683d5dccce898f16bb48071f08f2304c8ba09",
"0x7cc549cdef7248c11a84d718fd25eafb57d8d614",
"0xe31a9498a22493ab922bc0eb240313a46525ee0a",
"0xa294cca691e4c83b1fc0c8d63d9a3eef0a196de1",
"0x60009b78da046ac64ef789c29ca05b79cdf73c10"
]
target_list = []
for row in res:
    if row[0] not in alameda_address_list:
        target_list.append(row[0])

print(len(target_list))

970


In [55]:
interaction_sql = """
INSERT INTO dm.dm_acct_eoa_interaction_aggr_di
SELECT t.account_address
    ,t.interact_account_address
    ,t.token_address
    ,t.symbol
    ,case when c.address is null then 1 else 0 end as is_eoa_address
    ,t.transfer_in_cnt
    ,t.transfer_out_cnt
    ,t.transfer_in_amt
    ,t.transfer_out_amt
    ,t.transfer_in_dollar_amt
    ,t.transfer_out_dollar_amt
    ,t.updated_time
    ,t.block_number
    ,t.etl_time
FROM
(
    SELECT IFNULL(aa.account_address,bb.account_address) as account_address
        ,IFNULL(aa.interact_account_address,bb.interact_account_address) as interact_account_address
        ,IFNULL(aa.token_address,bb.token_address) as token_address
        ,IFNULL(aa.symbol,bb.symbol) as symbol
        ,in_cnt as transfer_in_cnt
        ,out_cnt as transfer_out_cnt
        ,in_amount as transfer_in_amt
        ,out_amount as transfer_out_amt
        ,in_dollar_amt as transfer_in_dollar_amt
        ,out_dollar_amt as transfer_out_dollar_amt
        ,GREATEST(aa.updated_time,bb.updated_time) as updated_time
        ,GREATEST(aa.block_number,bb.block_number) as block_number
        ,now() as etl_time
    FROM
    (
        SELECT a.to_address as account_address
            ,a.from_address as interact_account_address
            ,a.token_address
            ,max(a.transaction_hash) as transaction_hash
            ,max(a.symbol) as symbol
            ,count(*) as in_cnt
            ,sum(a.amount) as in_amount
            ,sum(a.amount * b.price) as in_dollar_amt
            ,max(transfer_time) as updated_time
            ,max(block_number) as block_number
        FROM dw.dwb_token_transfer_detail_eth_hi as a
        LEFT JOIN dw.dwb_token_price_eth_byday_di as b
        ON a.token_address = b.token_address
        AND date(a.transfer_time) = b.dt
        AND b.is_unusual = 0
        WHERE a.source != 'transaction'
        AND a.to_address in ({target_address_str})
        AND a.symbol is not null
        GROUP BY 1,2,3
        ORDER BY in_dollar_amt desc
    ) as aa
    FULL OUTER JOIN 
    (
        SELECT a.from_address as account_address
            ,a.to_address as interact_account_address
            ,a.token_address
            ,max(a.symbol) as symbol
            ,count(*) as out_cnt
            ,sum(a.amount) as out_amount
            ,sum(a.amount * b.price) as out_dollar_amt
            ,max(transfer_time) as updated_time
            ,max(block_number) as block_number
        FROM dw.dwb_token_transfer_detail_eth_hi as a
        LEFT JOIN dw.dwb_token_price_eth_byday_di as b
        ON a.token_address = b.token_address
        AND date(a.transfer_time) = b.dt
        AND b.is_unusual = 0
        WHERE source != 'transaction'
        AND from_address in ({target_address_str})
        GROUP BY 1,2,3
    ) as bb
    ON aa.account_address = bb.account_address
    AND aa.interact_account_address = bb.interact_account_address
    AND aa.token_address = bb.token_address
) as t
LEFT JOIN prod.contracts as c
ON t.interact_account_address = c.address
""".format(
    target_address_str = "'"+"','".join(target_list)+"'"
)


In [57]:
res = getSqlResult(interaction_sql)